<a href="https://colab.research.google.com/github/GustavoGatti/BreastCancerSiameseNeuralNetwork/blob/main/Sia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==2.0.0-rc0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, Lambda, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from keras.regularizers import l2

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from numpy import mean
from numpy import std
from numpy import ndarray
import numpy.random as rng
import random
from keras.layers import add

ImportError: ignored

In [ ]:
from keras import backend as K
import keras as KER

In [ ]:
from keras.layers import merge

In [ ]:
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
df.head()

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1,Classification
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114,0
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786,0
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697,0
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220,0
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920,0


In [ ]:
z = pd.DataFrame(data=df)
z.drop('Classification', inplace=True, axis=1)
z

,Age,BMI,Glucose,Insulin,HOMA,Leptin,Adiponectin,Resistin,MCP.1
0,48,23.500000,70,2.707,0.467409,8.8071,9.702400,7.99585,417.114
1,83,20.690495,92,3.115,0.706897,8.8438,5.429285,4.06405,468.786
2,82,23.124670,91,4.498,1.009651,17.9393,22.432040,9.27715,554.697
3,68,21.367521,77,3.226,0.612725,9.8827,7.169560,12.76600,928.220
4,86,21.111111,92,3.549,0.805386,6.6994,4.819240,10.57635,773.920
...,...,...,...,...,...,...,...,...,...
111,45,26.850000,92,3.330,0.755688,54.6800,12.100000,10.96000,268.230
112,62,26.840000,100,4.530,1.117400,12.4500,21.420000,7.32000,330.160
113,65,32.050000,97,5.730,1.370998,61.4800,22.540000,10.33000,314.050
114,72,25.590000,82,2.820,0.570392,24.9600,33.750000,3.27000,392.460


In [ ]:
g = df.Classification
g

0      0
1      0
2      0
3      0
4      0
      ..
111    1
112    1
113    1
114    1
115    1
Name: Classification, Length: 116, dtype: int64

In [ ]:
z.shape

(116, 9)

In [ ]:
z_train, z_test, g_train, g_test = train_test_split(z,g, test_size = 0.2,random_state = 0, stratify = g)

z_train = np.asarray(z_train)
g_train = np.asarray(g_train)
z_test = np.asarray(z_test)
g_test = np.asarray(g_test)
z_train

array([[5.40000000e+01, 3.60500000e+01, 1.19000000e+02, 1.19100000e+01,
        3.49598200e+00, 8.92700000e+01, 8.01000000e+00, 5.06000000e+00,
        2.18280000e+02],
       [5.30000000e+01, 3.67901662e+01, 1.01000000e+02, 1.01750000e+01,
        2.53493167e+00, 2.71841000e+01, 2.00300000e+01, 1.02630900e+01,
        6.95754000e+02],
       [7.20000000e+01, 2.55900000e+01, 8.20000000e+01, 2.82000000e+00,
        5.70392000e-01, 2.49600000e+01, 3.37500000e+01, 3.27000000e+00,
        3.92460000e+02],
       [5.10000000e+01, 1.91326531e+01, 9.30000000e+01, 4.36400000e+00,
        1.00110160e+00, 1.10816000e+01, 5.80762000e+00, 5.57055000e+00,
        9.06000000e+01],
       [3.80000000e+01, 2.33400000e+01, 7.50000000e+01, 5.78200000e+00,
        1.06967000e+00, 1.52600000e+01, 1.79500000e+01, 9.35000000e+00,
        1.65020000e+02],
       [4.50000000e+01, 2.31404959e+01, 1.16000000e+02, 4.90200000e+00,
        1.40262560e+00, 1.79973000e+01, 4.29470500e+00, 5.26330000e+00,
        5.1

In [ ]:
def cosine(vector):
  x, y = vector
  dotx = K.sum(x*x, axis=1,keepdims=True)
  doty = K.sum(y*y, axis=1,keepdims=True)
  dotxy = K.sum(x*y, axis=1,keepdims=True)
  cos = (dotxy/(K.sqrt(dotx) * K.sqrt(doty)))
  return cos

In [ ]:
def eucl_dist_output_shape(shapes):
  shape1, shape2 = shapes
  return shape1[0], 1

In [ ]:
def contrastive_loss(y_true,y_pred):
  return K.mean(y_true*(2*K.exp(-(1/K.square(y_pred-1.2)))) + (1 - y_true)*(2*K.exp(-(1/K.square(y_pred+1.2)))))

In [ ]:
def logistic_loss(y_true,y_pred):
  return K.mean(K.log(1 * K.exp(-1*y_pred*y_true)))

In [ ]:
def compute_accuracy(y_true,y_pred):
  classified = K.round(y_pred)
  return K.mean(K.equal(classified, y_true))

In [ ]:
def compute_final_accuracy(predictions, labels, print_prediction = False):
  if print_prediction:
    print(predictions)

  classified = np.round(predictions.ravel())
  return np.mean(classified == labels)

In [ ]:
def difference(vector):
  x, y = vector
  return x-y

In [ ]:
def cos_distance(vector):
  return 1-cosine(vector)

In [ ]:
def distance(cos):
  return 1 - K.abs(cos)

In [ ]:
def create_base_network(input_shape):
  model = Sequential()
  model.add(keras.layers.Reshape(input_shape = (9, 1), target_shape=(9,1)))
  model.add(Conv1D(filters=32, kernel_size=2, activation = 'relu', input_shape = (9,1)))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Conv1D(filters=64, kernel_size=2, activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(64, activation = 'relu'))
  model.add(Dropout(0.5))

  model.add(Dense(1, activation = 'sigmoid'))
  
  return model

In [ ]:
def execute(epochs=2000):
  np.random.seed(100)
  random.seed(100)
  
  #dataset

  cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
  x, x1, y, y1, scaler = Transformacao(cancer)
  x_train, y_train = Montando_data_set(x,x1,y,y1)

  cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
  x12, x123, y12, y123 = Transformacao_teste(cancer)
  x_test, y_test = Montando_data_set_test(x,x123,y,y123)  


  input = (9,1)
  base_network_w = create_base_network(input)
  left_input = Input(shape=(input))
  right_input = Input(shape=(input)) 

  left = base_network_w(left_input)
  right = base_network_w(right_input)

  #both = merge([left, right], mode = cosine, output_shape = eucl_dist_output_shape)
  both = tf.keras.layers.Dot(axes=1)([left, right])

  prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(both)

  model = Model(inputs=[left_input,right_input],outputs=prediction)

  loss = contrastive_loss
  optimizer = Adam(0.0005)
  #model.compile(loss=loss, optimizer = optimizer, metrics = [contrastive_loss, compute_accuracy])
  model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])

  hist = model.fit(x_train, y_train, epochs=epochs,validation_data=(x_test,y_test), verbose=1)

  return hist,model, x_test, y_test


In [ ]:
hist, model,x_test, y_test = execute()

NameError: ignored

In [ ]:
def plot_learningCurve(history, epochs):

  #Plot Training e validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

  #Plot training e validation accuracy loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(hist, 2000)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)